In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# #이후 런타임 다시 시작

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,532 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and d

In [ ]:
import warnings   
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams["font.size"] = 12
plt.rcParams['xtick.labelsize'] = 12.
plt.rcParams['ytick.labelsize'] = 12.
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score

In [ ]:
cd /content/drive/MyDrive/UBION_final_project/data

/content/drive/.shortcut-targets-by-id/1cbruNkx_Sh7hGut3yOs6kDVmEN0tQAuf/UBION_final_project/data


# show_result, store_result

In [ ]:
def show_result(y_test, prediction):
  global condition
  CM = confusion_matrix(y_test, prediction) #Confusion_matrix
  A = round( accuracy_score(y_test,prediction),3)  # 정확도
  P = round( precision_score(y_test, prediction),3)  #정밀도
  R = round( recall_score(y_test, prediction), 3) #재현율
  F = round( f1_score(y_test, prediction), 3) #f1-score
  
  condition.append(CM)
  condition.append(A)
  condition.append(P)
  condition.append(R)
  condition.append(F)

In [ ]:
# final_Lasso_result = pd.DataFrame(columns = ['데이터프레임', 'train_test_split(Y/R)', 't-test(Y/N)', '변수선택법', 'VIF(Y/N)','sampling 방식', '적용모델','선택된 변수','혼동행렬', 'Accuracy', 'Precision', 'Recall', 'F1-score','hyperparameter 변경'])
# final_Lasso_result.to_csv('final_Lasso_result.csv', index=False)

In [ ]:
def store_result(condition):
  final_Lasso_result = pd.read_csv('final_Lasso_result.csv')

  condition= pd.DataFrame({len(final_Lasso_result): condition}).transpose()
  condition.columns = ['데이터프레임', 'train_test_split(Y/R)', 't-test(Y/N)', '변수선택법', 'VIF(Y/N)','sampling 방식', '적용모델','선택된 변수','혼동행렬', 'Accuracy', 'Precision', 'Recall', 'F1-score','hyperparameter 변경']
  
  final_Lasso_result = pd.concat([final_Lasso_result, condition], axis=0)

  final_Lasso_result.to_csv('final_Lasso_result.csv', index=False)

# t-test

In [ ]:
def z_test(eq_var, nc, bc):
    from statsmodels.stats.weightstats import ztest
    sig_diff = []

    for i in eq_var:
        ser_1 = nc[i]
        ser_2 = bc[i]

        z_result = np.round(ztest(ser_1, ser_2)[1],3)
        if z_result < 0.01:
            sig_diff.append(i)
        else:
            pass
    return sig_diff

In [ ]:
def welch_t_test(neq_var,nc_,bc_):
    from scipy.stats import ttest_ind

    sig_diff1 = []

    for i in neq_var:
        ser_1 = nc_[i]
        ser_2 = bc_[i]

        wt_result = np.round(ttest_ind(ser_1, ser_2, equal_var=False)[1],3)
        if wt_result < 0.01:
            sig_diff1.append(i)
        else:
            pass
    return sig_diff1

In [ ]:
def Excute_t_test(X_train, y_train):
    from scipy.stats import bartlett
    
    feature_list = X_train.columns.to_list()
    
    subset= X_train
    subset['차기부도여부'] = y_train

    bc_df = subset[subset['차기부도여부']==1]
    nc_df = subset[subset['차기부도여부']==0]

    #등분산, 이분산성 검정
    eq_var = []
    neq_var = []
    for i in subset.columns[:-1]:
        ser_1 = nc_df[i]
        ser_2 = bc_df[i]

        lresult = np.round(bartlett(ser_1, ser_2)[1],3)
        if lresult < 0.05:
            eq_var.append(i) #등분산
        else:
            neq_var.append(i) #이분산

    
    sig_diff = z_test(eq_var, nc_df, bc_df) #등분산 -> z_test
    sig_diff1 = welch_t_test(neq_var, nc_df, bc_df) #이분산 -> t_test

    sig_val = sig_diff + sig_diff1

    return sig_val

# Excute_model

## basic

In [ ]:
def Excute_lg (X_train, y_train, X_test, y_test):
  lr = LogisticRegression(random_state=0)
  
  lr.fit(X_train, y_train) 
  prediction = lr.predict(X_test)

  show_result(y_test, prediction)

In [ ]:
def Excute_sgd(X_train, y_train, X_test, y_test):
  sgd = SGDClassifier(random_state=0)

  sgd.fit(X_train, y_train)
  prediction = sgd.predict(X_test)
  
  show_result(y_test, prediction)

In [ ]:
def Excute_knn(X_train, y_train, X_test, y_test):
  knn = KNeighborsClassifier()

  knn.fit(X_train, y_train)
  prediction = knn.predict(X_test)

  show_result(y_test, prediction)

In [ ]:
def Excute_svm(X_train, y_train, X_test, y_test):
  svc = SVC(random_state=0)

  svc.fit(X_train, y_train)
  prediction = svc.predict(X_test)

  show_result(y_test, prediction)

In [ ]:
def Excute_dt(X_train, y_train, X_test, y_test):
  dt = DecisionTreeClassifier(random_state=0)

  dt.fit(X_train, y_train)
  prediction = dt.predict(X_test)

  show_result(y_test, prediction)

In [ ]:
def Excute_rf(X_train, y_train, X_test, y_test):
  rfc = RandomForestClassifier(random_state=0)

  rfc.fit(X_train, y_train)
  prediction = rfc.predict(X_test)

  show_result(y_test, prediction)

In [ ]:
def Excute_pr(X_train, y_train, X_test, y_test):
  dt2 = DecisionTreeClassifier(max_depth=2)

  dt2.fit(X_train, y_train)
  prediction = dt2.predict(X_test)
  
  show_result(y_test, prediction)

## change_cutoff

In [ ]:
def change_cutoff(result_df):
  value_00 = result_df[(result_df['actual']==0) & (result_df['prediction']== 0)]['부도_prob'].max()
  value_01 = result_df[(result_df['actual']==0) & (result_df['prediction']== 1)]['부도_prob'].min()
  value_10 = result_df[(result_df['actual']==1) & (result_df['prediction']== 0)]['부도_prob'].max()
  value_11 = result_df[(result_df['actual']==1) & (result_df['prediction']== 1)]['부도_prob'].min()

  if value_10 < value_00:
    cutoff = value_11
    result_df.loc[(result_df['actual']==0) & (result_df['prediction']== 1) & (result_df['부도_prob']<cutoff) , 'prediction'] = 0
  
  if value_11 > value_01:
    cutoff= value_00
    result_df.loc[(result_df['actual']==1) & (result_df['prediction']== 0) & (result_df['부도_prob']>cutoff) , 'prediction'] = 1

  return result_df

In [ ]:
def Excute_lg_change_cutoff (X_train, y_train, X_test, y_test):
  lr = LogisticRegression(random_state=0, probability=True)
  lr.fit(X_train, y_train) 
  prediction = lr.predict(X_test)
  prediction_prob =lr.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

In [ ]:
def Excute_sgd_change_cutoff(X_train, y_train, X_test, y_test):
  sgd = SGDClassifier(random_state=0)
  sgd.fit(X_train, y_train)
  prediction = sgd.predict(X_test)
  prediction_prob =sgd.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

In [ ]:
def Excute_svm_change_cutoff(X_train, y_train, X_test, y_test):
  svc = SVC(random_state=0, probability=True)
  svc.fit(X_train, y_train)
  prediction = svc.predict(X_test)
  prediction_prob =svc.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

In [ ]:
def Excute_pr_change_cutoff(X_train, y_train, X_test, y_test):
  dt2 = DecisionTreeClassifier(max_depth=2)
  dt2.fit(X_train, y_train)
  prediction = dt2.predict(X_test)
  prediction_prob =dt2.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

In [ ]:
def Excute_dt_change_cutoff(X_train, y_train, X_test, y_test):
  dt = DecisionTreeClassifier(random_state=0)
  dt.fit(X_train, y_train)
  prediction = dt.predict(X_test)
  prediction_prob =dt.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

In [ ]:
def Excute_knn_change_cutoff(X_train, y_train, X_test, y_test):
  knn = KNeighborsClassifier()
  knn.fit(X_train, y_train)
  prediction = knn.predict(X_test)
  prediction_prob =knn.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

In [ ]:
def Excute_rf_change_cutoff(X_train, y_train, X_test, y_test):
  rfc = RandomForestClassifier(random_state=0)
  rfc.fit(X_train, y_train)
  prediction = rfc.predict(X_test)
  prediction_prob =rfc.predict_proba(X_test)

  result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
  result_df['actual'] = y_test
  result_df['prediction'] = prediction

  return_df = change_cutoff(result_df)
  show_result(result_df['actual'], result_df['prediction'])

# sampling

In [ ]:
def RUSE(X_train, y_train):
  from imblearn.under_sampling import RandomUnderSampler
  rus = RandomUnderSampler()
  
  X_undersampled, y_undersampled = rus.fit_resample(X_train, y_train)
  
  return X_undersampled, y_undersampled

In [ ]:
def ROSE(X_train, y_train):
  from imblearn.over_sampling import RandomOverSampler
  ros = RandomOverSampler()

  X_oversampled, y_oversampled = ros.fit_resample(X_train, y_train)
  
  return X_oversampled, y_oversampled

In [ ]:
def SMOTE(X_train, y_train):
  from imblearn.over_sampling import SMOTE
  smote = SMOTE()
  X_smoted, y_smoted = smote.fit_resample(X_train,y_train)
  
  return X_smoted, y_smoted

# feature_selection

In [ ]:
def lasso_selection_all(X, y):
    def rmse_cv(model):
        rmse= np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = 5))
        return(rmse)
    model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005]).fit(X, y)
    
    # print('rmse_cv: %f' %(rmse_cv(model_lasso).mean()))
    coef = pd.Series(model_lasso.coef_, index = X.columns)
    # print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

    imp_coef = pd.concat([coef.sort_values().head(),
                        coef.sort_values().tail()])
    
    lasso_result = pd.DataFrame({
        'feature' : coef.index,
        'coef': coef
    })
    lasso_variables = lasso_result[lasso_result['coef']!=0].index.tolist()
    return lasso_variables

In [ ]:
def filter_vif(X_train, variables):
    from patsy import dmatrices
    import statsmodels.api as sm
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    vif = pd.DataFrame()
    vif['VIF'] = [variance_inflation_factor(X_train[variables].values, i) for i in range(X_train[variables].shape[1])]
    vif['feature'] = X_train[variables].columns
    
    non_vif_features = vif.loc[vif['VIF'] <= 10]['feature']
    return non_vif_features

# train/test/split

In [ ]:
def split_by_Y(df):
    after_2018  = df.loc[df['회계년도'] >= 2018]['차기부도여부'].value_counts()[1]
    before_2018 = df.loc[df['회계년도'] <  2018]['차기부도여부'].value_counts()[1]
    SIZE = after_2018/len(df)

    test = df[df['회계년도'] >= 2018]
    train = df[df['회계년도'] < 2018]
    
    test.drop(columns = ['회계년도','회사명', '거래소코드','산업대분류'], inplace=True)
    train.drop(columns= ['회계년도','회사명', '거래소코드','산업대분류'], inplace=True)

    X_train = train.drop(['차기부도여부'], axis= 1)
    y_train = train['차기부도여부']

    X_test = test.drop(['차기부도여부'], axis=1)
    y_test = test['차기부도여부']

    return X_train, y_train, X_test, y_test

In [ ]:
def split_by_R(df):
    from sklearn.model_selection import train_test_split
    
    data = df.drop(['차기부도여부'], axis= 1)
    target = df['차기부도여부']

    # train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.178, shuffle=True, stratify=target, random_state=34)
    X_test.drop(columns = ['회계년도','회사명', '거래소코드','산업대분류'], inplace=True)
    X_train.drop(columns= ['회계년도','회사명', '거래소코드','산업대분류'], inplace=True)

    return X_train, y_train, X_test, y_test

# main

In [ ]:
def RUN_ALL(filename, train_test_split_method, tt, feature_selection_method, vv, sampling_method, model_name):
    df = pd.read_csv(filename+'.csv')
    global condition

    if train_test_split_method == 'Y' :
        X_train, y_train, X_test, y_test = split_by_Y(df)
    elif train_test_split_method == 'R' :
        X_train, y_train, X_test, y_test = split_by_R(df)

    if tt =='t_Y':
        variable = Excute_t_test(X_train, y_train)
    elif tt =='t_N':
        variable = X_train.columns.tolist()

    try: #에러가 발생될 지도 모르는 코드
        if feature_selection_method == 'Lasso':
            selected_variables_list = lasso_selection_all(X_train[variable], y_train)

    except: #에러가 나면 try 내의 구문대신 실행될 구문
        selected_variables_list = variable
        condition[3]= 'error'

    finally: #에러 유무와 상관없이 실행될 구문
        if vv =='v_Y':
            selected_variables_list = filter_vif(X_train, selected_variables_list)
            if len(selected_variables_list)==0:
                return 'F';
            condition.append(selected_variables_list.tolist())
        elif vv =='v_N':
            condition.append(selected_variables_list)

        X_train = X_train[selected_variables_list]
        X_test = X_test[selected_variables_list]

        if sampling_method == 'RUSE' :
            sampled_X_train, sampled_y_train = RUSE( X_train, y_train )
        elif sampling_method =='ROSE' :
            sampled_X_train, sampled_y_train = ROSE( X_train, y_train )
        elif sampling_method =='SMOTE' :
            sampled_X_train, sampled_y_train = SMOTE( X_train, y_train)
        elif sampling_method =='None':
            sampled_X_train, sampled_y_train = X_train, y_train


        if model_name =='lg':
            Excute_lg(sampled_X_train, sampled_y_train, X_test, y_test)
        elif model_name =='sgd':
            Excute_sgd(sampled_X_train, sampled_y_train, X_test, y_test)
        elif model_name =='knn':
            Excute_knn(sampled_X_train, sampled_y_train, X_test, y_test)
        elif model_name =='svm':
            Excute_svm(sampled_X_train, sampled_y_train, X_test, y_test)
        elif model_name =='dt':
            Excute_dt(sampled_X_train, sampled_y_train, X_test, y_test)
        elif model_name =='pr':
            Excute_pr(sampled_X_train, sampled_y_train, X_test, y_test)
        elif model_name =='rf':
            Excute_rf(sampled_X_train, sampled_y_train, X_test, y_test)
    return 'T'

In [ ]:
for df in ['basic_final_df','diff_df','binning_final_df','small_business_df']:
    for split in ['Y', 'R']:
        for tt in ['t_Y']:
            for fs in ['Lasso']:
                for vv in ['v_Y', 'v_N']:
                    for sm in ['RUSE','ROSE','SMOTE','None']:
                        for m in ['lg','sgd','knn','svm','dt','pr','rf']:
                            condition = [df, split, tt, fs, vv, sm, m]
                            print("***", condition)
                            Re = RUN_ALL(condition[0], condition[1], condition[2], condition[3], condition[4], condition[5], condition[6])
                            if Re=='T':
                                condition.append('none') # parameter 조정 X
                                store_result(condition)

*** ['basic_final_df', 'Y', 't_Y', 'Lasso', 'v_Y', 'RUSE', 'lg']


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 63.47798082140122, tolerance: 0.013936087064061405
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 63.40126752756457, tolerance: 0.013936087064061405
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to in

*** ['basic_final_df', 'Y', 't_Y', 'Lasso', 'v_Y', 'RUSE', 'sgd']


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 63.47798082140122, tolerance: 0.013936087064061405
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 63.40126752756457, tolerance: 0.013936087064061405
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to in

*** ['basic_final_df', 'Y', 't_Y', 'Lasso', 'v_Y', 'RUSE', 'knn']


...

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:2241: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03411299196736195, tolerance: 0.013660819509779697
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.63660477664992, tolerance: 0.013660819509779697
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.60694808409258, tolerance: 0.013660819509779697
  positive,
/usr/loc

*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'None', 'pr']


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:2241: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03411299196736195, tolerance: 0.013660819509779697
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.63660477664992, tolerance: 0.013660819509779697
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.60694808409258, tolerance: 0.013660819509779697
  positive,
/usr/loc

*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'None', 'rf']


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:2241: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03411299196736195, tolerance: 0.013660819509779697
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.63660477664992, tolerance: 0.013660819509779697
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.60694808409258, tolerance: 0.013660819509779697
  positive,
/usr/loc